## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## 2. Read the CSV

In [2]:
df_species = pd.read_csv('assets/species.csv', usecols=['Park Name', 'Category', 'Common Names', 'Occurrence', 'Nativeness', 'Abundance', 'Conservation Status'])

## 3. Clean the data

In [5]:
# check for whitespace in column names
list(df_species.columns)

['Park Name',
 'Category',
 'Common Names',
 'Occurrence',
 'Nativeness',
 'Abundance',
 'Conservation Status']

In [10]:
df_species.dtypes

Park Name              object
Category               object
Common Names           object
Occurrence             object
Nativeness             object
Abundance              object
Conservation Status    object
dtype: object